**Auxiliary percentage = 20% so far**

---
**Problems to solve:**

1. 5 problems with 2 variables for which you can easily find the solution geometrically (for LPs you should know the solution exactly, i.e. in which vertex it is, while for QPs you should the solution at least approximately).

2. 5 problems with more (5-10) variables.

The same applies to both, linear programs (LPs) and QPs. You can use the same feasible set for both and just choose a different objective function. Note, however, that even though the feasible set is the same for LPs and QPs (i.e. the constraints are linear), we used a different format for each. You can e.g. start with the constraints $x>=0$ and $a_i^T x <= b_i$ to fit the QP format.

To make things easier, you can always choose the feasible set for which you know a feasible point that can be used as a starting point $x_0$. Particularly, for the constraints given via $x >= 0$ and $a_i^T x <= b_i$, point $x=0$ is always feasible if $b_i >= 0$.

**When is the problem "solved"?**

Since we are only dealing with LPs and QPs, the algorithm should be stopped if the multipliers have the required signs, signaling that the KKT conditions are satisfied.

---

## The simplex method

### LP no.1 with 2 variables

Maximize

$$
z = 4x + 5y
$$

s.t.

$$
x + y <= 20\\
3x + 4y <= 72
$$

### Analytical

<img src="problem1.jpg" alt="drawing" width="600"/>

### Programmatic

...but first I need to add slack variables and stuff to get to the normal form (I will convert it back programmatically later, so this is just for my sanity). Normal form states:

> Maximize the value of $c^T x$ among all vectors $x$ in $R^n$ satisfying $Ax <= b$.

In [1]:
c = [4, 5, 0, 0]  # Singifying objective: 4x_1 + 5x_2 + 0x_3 + 0x_4, where x_3 and x_4 are slack variables
A = [  # Constraints matrix
    [1, 1, 1, 0],  # First constraint
    [3, 4, 0, 1],  # Second constraint
]
b = [20, 72]  # Right side of the constraints

In [2]:
import numpy as np

def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

In [3]:
def can_be_improved(tableau):
    """
    Check where nonbasic values can be increased without making the objective function value smaller.
    """
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

In [4]:
def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(np.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

In [5]:
def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

In [6]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

In [7]:
def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

In [8]:
solution = simplex(c, A, b)
print('solution: ', solution)

solution:  [8.0, 12.0, 0, 0, 0]


Yay!!! It's the same {8, 12} point! 10% done...

### LP no.2 with 2 variables

Maximize $x_1 + x_2$

s.t.

$$
x_1 ≥ 0\\
x_2 ≥ 0\\ 
-x_1 + x_2 ≤ 2\\
x_1 ≤ 4\\
x_2 ≤ 4
$$

This is actually just taken from [here](https://radzion.com/blog/operations/simplex).

In [10]:
c = [1, 1, 0, 0, 0]
A = [
    [-1, 1, 1, 0, 0],
    [ 1, 0, 0, 1, 0],
    [ 0, 1, 0, 0, 1]
]
b = [2, 4, 4]

solution = simplex(c, A, b)
print('solution: ', solution)

solution:  [4.0, 4.0, 2.0, 0, 0, 0]
